<a href="https://colab.research.google.com/github/JohnsonYu0924/114_2_text-analysis/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A/script/%E8%99%95%E7%90%86%E9%80%90%E5%AD%97%E7%A8%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0：環境設定

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json, re, os, csv
import re

os.getcwd()

'/content'

# STEP 1：抓取逐字稿

In [ ]:
# 先寫看看

with open('191218_政見發表會.md', 'r', encoding='utf-8') as t:
    transcript = t.read()
    print(type(transcript)) #class

senten = []
transcript = re.split(r"\n", transcript)
for i in transcript:
    if i != "":
        senten.append(i.strip())

candidate = input("要查詢的人：")
target_candidate = '#### ' + candidate
candidateTranscript = []
is_collecting = False

for front, later in zip(senten[:-1], senten[1:]):
  if front == target_candidate:
    is_collecting = not is_collecting
  if is_collecting and not re.search(r"###", later):
    candidateTranscript.append(later)
    # print(later)
  elif is_collecting and re.search(r"###", later):
    is_collecting = not is_collecting

for i in candidateTranscript:
  print(i)


In [ ]:
# 請Gemini幫我變成def好處理

import re
import os

def candiTrans(filename, candidate):
    """
    從指定的 Markdown 檔案中提取特定候選人的發言內容。

    發言區段的判斷邏輯是：
    1. 遇到 '#### [候選人姓名]' 標記，開啟/關閉收集狀態 (is_collecting)。
    2. 在收集狀態下，直到下一個包含 '###' 的行 (代表另一位候選人發言的開始)
       出現為止，收集該區間的所有內容。

    Args:
        filename (str): 要讀取的檔案名稱。
        candidate (str): 要查詢的候選人姓名 (例如: '韓國瑜' 或 '蔡英文')。

    Returns:
        list: 包含該候選人所有發言句子的列表。
    """

    # 檢查檔案是否存在
    if not os.path.exists(filename):
        print(f"錯誤：檔案 '{filename}' 不存在。")
        return []

    # 1. 檔案讀取與文本清洗
    try:
        with open(filename, 'r', encoding='utf-8') as t:
            transcript = t.read()
    except Exception as e:
        print(f"讀取檔案時發生錯誤: {e}")
        return []

    # 將文本按換行符分割並清除空白行和行首尾空白
    transcript_lines = re.split(r"\n", transcript)
    senten = []
    for line in transcript_lines:
        stripped_line = line.strip()
        if stripped_line != "":
            senten.append(stripped_line)

    # 2. 設定目標分隔符號
    target_candidate = '#### ' + candidate
    candidateTranscript = []
    is_collecting = False

    # 3. 遍歷並提取發言內容 (使用 zip 進行前/後配對)
    # 注意: 這個邏輯不包含區段開頭和結尾的分隔符號本身。
    # 並且，這裡假設所有分隔符號都至少包含 '###'。
    for front, later in zip(senten[:-1], senten[1:]):

        # 檢查是否遇到目標候選人發言的開頭 (例如: '#### 韓國瑜')
        if front == target_candidate:
            # 切換狀態：False -> True (開始收集) 或 True -> False (結束收集)
            is_collecting = not is_collecting

        # 如果正在收集且下一句不是分隔符號
        if is_collecting and not re.search(r"###", later):
            candidateTranscript.append(later)

        # 如果正在收集且下一句是分隔符號 (結束收集)
        elif is_collecting and re.search(r"###", later):
            # 切換狀態：True -> False (停止收集)
            is_collecting = not is_collecting

    return candidateTranscript

In [ ]:
# 調用function

file_name = input("請輸入檔案名稱：")
file_name = file_name + '.md'

candidate_name = input("請輸入要查詢的人：")

result_list = candiTrans(file_name, candidate_name)

if result_list:
    for item in result_list:
        print(item)
else:
    print("未找到發言內容。")


# STEP 3：導出為.json檔

In [ ]:
jsonDataName = input('輸入檔名（eg. 191218_Tsai/Han/Soong): ')
jsonDataName = jsonDataName + ".json"

with open(jsonDataName, 'w', encoding='utf-8') as d:
  json.dump(result_list, d, ensure_ascii=False, indent=4)

#重複做把所有.md檔整理成.json

輸入檔名（eg. 191218_Tsai/Han/Soong): 191229_Soong


# STEP 4：清理逐字稿

In [ ]:
def basicCleaner(file_name):
  # file_name = '191227_Han.json'
  file_name_w = re.sub(r"\.json", "", file_name)

  with open(file_name, 'r', encoding='utf-8') as f:
    transcript = json.load(f)
    print(transcript)

  cleanTrans = []
  for i in transcript:
    i = re.sub(r"https://\S+", " ", i)
    i = re.sub(r"[!\[\]:「」『』《》〈〉（）\(\)，、：；。\.]", " ", i)
    # i = re.sub(r"\d+", "", i) #可以考慮要不要處理數字，跟情緒沒什麼關係
    i = i.strip().lower()
    cleanTrans.append(i)

  for i in cleanTrans:
    print(i)

  file_name_w = file_name_w + '_cleaned.json'
  with open(file_name_w, 'w', encoding='utf-8') as f:
    json.dump(cleanTrans, f, ensure_ascii=False, indent=4)


In [ ]:
# 調用function

# fileName = input("請輸入檔案名稱（含.json）：")
# basicCleaner(fileName)

# 批次處理
path = "/content/"
print(os.getcwd())
dirList = [i for i in os.listdir(path) if i.endswith(".json")]
if len(dirList) == 0:
  print("There's no '.json' data.")
else:
  print(dirList)

for i in dirList:
  basicCleaner(i)

In [ ]:
#刪除資料

import os
import glob

# ⚠️ 請將這裡替換成您要清理的資料夾路徑
folder_path = '/content'

# 定義要刪除的檔名模式
patterns_to_delete = ['*_cleaned.json', '*_cleaned_cleaned.json']

# 遍歷所有模式並刪除檔案
deleted_count = 0
for pattern in patterns_to_delete:
    # 組合完整的搜索路徑
    search_path = os.path.join(folder_path, pattern)

    # glob.glob() 找到所有符合模式的檔案路徑
    for file_path in glob.glob(search_path):
        try:
            # 刪除檔案
            os.remove(file_path)
            print(f"✅ 成功刪除: {file_path}")
            deleted_count += 1
        except OSError as e:
            # 處理刪除失敗的情況（例如權限不足）
            print(f"❌ 錯誤：無法刪除 {file_path}. {e}")

print(f"\n--- 總結：共刪除了 {deleted_count} 個檔案。 ---")

✅ 成功刪除: /content/191227_Soong_cleaned.json
✅ 成功刪除: /content/191218_Tsai_cleaned.json
✅ 成功刪除: /content/191227_Han_cleaned.json
✅ 成功刪除: /content/191229_Han_cleaned.json
✅ 成功刪除: /content/191229_Soong_cleaned.json
✅ 成功刪除: /content/191225_Han_cleaned.json
✅ 成功刪除: /content/191227_Tsai_cleaned.json
✅ 成功刪除: /content/191218_Han_cleaned.json
✅ 成功刪除: /content/191229_Tsai_cleaned.json
✅ 成功刪除: /content/191225_Soong_cleaned.json
✅ 成功刪除: /content/191218_Soong_cleaned.json
✅ 成功刪除: /content/191225_Tsai_cleaned.json

--- 總結：共刪除了 12 個檔案。 ---


# STEP 5：切token
利用中研院「CKIP Transformers」代替「jieba」進行token處理

In [5]:
# !pip install -U ckip-transformers

# import tools: token(WS), tagging(POS), 實體辨識(NER)
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

# diver initiation(初始化一次之後先關一下)
WS_driver = CkipWordSegmenter(model='bert-base')
POS_driver = CkipPosTagger(model='bert-base')
NER_driver = CkipNerChunker(model='bert-base')

# imput data
import os, json
with open("/content/191218_Han_cleaned.json", "r", encoding="utf-8") as f:
  f = json.load(f)
  myText = "".join(f)
print(myText[:5])

# run
myText_WS = WS_driver(myText)
myText_POS = POS_driver(myText_WS) # Changed to pass myText_WS to POS_driver
myText_NER = NER_driver(myText)

# print result
print("\nWS \n", myText_WS)
print("POS \n", myText_POS)
print("NER \n", myText_NER)


# 引入stopwords dictionary
with open("/content/stopwords.txt", "r", encoding="utf-8") as f:
  stopwords = f.read().split("\n")
  stopwords = set(stopwords)

# filtered = []
# for i in range(len(myText_WS)):



KeyboardInterrupt: 

# 暫存區

## Question: 能不能把上面操作變成批次處理？（好瘋）

In [ ]:
# 從HTHL清資料（先不用這個東西）

HTML1218 = open('1218.html', 'r', encoding='utf-8')
soup = BeautifulSoup(HTML1218, 'html.parser')
# print(soup)


all_spans = soup.find_all('span')
content_list = []
flag = False

for span in all_spans:
    text = span.get_text(strip=True)
    if text == '逐字稿：':
        flag = True
    if flag and text:
        clean_text = re.sub(r'[\u3000\s]+', ' ', text).strip()
        print(text)
    # if clean_text and clean_text != ' ':
    #     content_list.append(clean_text)

FileNotFoundError: [Errno 2] No such file or directory: '1218.html'

In [ ]:
# 列印token和tag的結果

def pack_ws_pos_sentece(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        res.append(f"{word_ws}({word_pos})")
    return "\u3000".join(res)

for myText, ws, pos in zip(myText, myText_WS, myText_POS):
  print(myText)
  print(pack_ws_pos_sentece(ws, pos))

NameError: name 'myText' is not defined